<a href="https://colab.research.google.com/github/navneetkrc/Colab_fastai/blob/master/YELP_review_ULMFiT_fastai_Short.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install Dependencies
Let's install PyTorch and fastai libraries first. You have to repeat this step every time you restart this notebook in colab.

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [0]:
import torch
print(torch.__version__)
import fastai
print(fastai.__version__) #1.0.38


1.0.0
1.0.39


In [0]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

## Import Dataset

###Install Kaggle Dependencies

In [0]:
!wget https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
!tar -xvzf yelp_review_polarity_csv.tgz
path = Path('yelp_review_polarity_csv');path


--2019-01-06 10:34:55--  https://s3.amazonaws.com/fast-ai-nlp/yelp_review_polarity_csv.tgz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.168.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.168.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166373201 (159M) [application/x-tar]
Saving to: ‘yelp_review_polarity_csv.tgz’

yelp_review_polarit 100%[===================>] 158.67M  68.2MB/s    in 2.3s    

2019-01-06 10:34:57 (68.2 MB/s) - ‘yelp_review_polarity_csv.tgz’ saved [166373201/166373201]

yelp_review_polarity_csv/
yelp_review_polarity_csv/train.csv
yelp_review_polarity_csv/readme.txt
yelp_review_polarity_csv/test.csv


PosixPath('yelp_review_polarity_csv')

In [0]:
train_csv = path/'train.csv'
train = pd.read_csv(train_csv, header=None)

train.columns = ['label', 'data']
train.head()

#train = pd.read_csv('input/drugsComTrain_raw.csv')
#test = pd.read_csv('input/drugsComTest_raw.csv')

,label,data
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [0]:
valid_csv = path/'test.csv'
valid = pd.read_csv(valid_csv, header=None)

valid.columns = ['label', 'data']
valid.head()

,label,data
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


## from_df       [source](https://github.com/fastai/fastai/blob/master/fastai/text/data.py#L157)
from_df(path:PathOrStr, train_df:DataFrame, valid_df:DataFrame, test_df:OptDataFrame=None, tokenizer:Tokenizer=None, vocab:Vocab=None, classes:StrList=None, text_cols:Union[int, Collection[int], str, StrList]=1, label_cols:Union[int, Collection[int], str, StrList]=0, label_delim:str=None, kwargs) → DataBunch

Create a [TextDataBunch](https://docs.fast.ai/text.data.html#TextDataBunch) from DataFrames.


This method will use train_df, valid_df and maybe test_df to build the TextDataBunch in path. You can specify text_cols and label_cols.**If there are several text_cols, the texts will be concatenated together with an optional field token.**
If there are several label_cols, the labels will be assumed to be one-hot encoded and classes will default to label_cols (you can ignore that argument for a language model). tokenizer will be used to parse those texts into tokens.

You can pass a specific vocab for the numericalization step (if you are building a classifier from a language model you fine-tuned for instance). kwargs will be split between the [TextDataset](https://docs.fast.ai/text.data.html#TextDataset) function and to the class initialization, you can precise there parameters such as max_vocab, chunksize, min_freq, n_labels (see the [TextDataset](https://docs.fast.ai/text.data.html#TextDataset) documentation) or bs, bptt and pad_idx (see the sections LM data and classifier data).



##Create Dataframe
This provides the Label/Target and the associated text

In [0]:
# create a dataframe
train_df = train
print(train_df.shape)
train_df.head()

(560000, 2)


,label,data
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [0]:
test_df = valid
print(test_df.shape)
test_df.head()

(38000, 2)


,label,data
0,2,"Contrary to other reviews, I have zero complai..."
1,1,Last summer I had an appointment to get new ti...
2,2,"Friendly staff, same starbucks fair you get an..."
3,1,The food is good. Unfortunately the service is...
4,2,Even when we didn't have a car Filene's Baseme...


In [0]:
df= train_df.append(test_df, ignore_index=True)


In [0]:
df.shape

(598000, 2)

In [0]:
df = df.reset_index(drop = True)
df.head()

,label,data
0,1,"Unfortunately, the frustration of being Dr. Go..."
1,2,Been going to Dr. Goldberg for over 10 years. ...
2,1,I don't know what Dr. Goldberg was like before...
3,1,I'm writing this review to give you a heads up...
4,2,All the food is great here. But the best thing...


In [0]:
df['label'].value_counts()

2    299000
1    299000
Name: label, dtype: int64

## Preprocessing

Tokenization

Remove StopWords
Detokenization

In [0]:
#df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")

#import nltk
#nltk.download('stopwords')

#from nltk.corpus import stopwords 
#stop_words = stopwords.words('english') 

# tokenization 
#tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
#tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words]) 

# de-tokenization 
#detokenized_doc = [] 
#for i in range(len(df)): 
#    t = ' '.join(tokenized_doc[i]) 
#    detokenized_doc.append(t) 
#df['text'] = detokenized_doc

In [0]:
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.0635451505, random_state = 12)

In [0]:
df_trn.shape, df_val.shape

((560000, 2), (38000, 2))

## Data Preparation

### Language model data
for learning the Language semantics

### Classifier model data
for learning the classification Task on text data

**text.learner contains helper functions to quickly create a language model or an RNN classifier**


In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val,path = "")

In [0]:
# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val,vocab=data_lm.train_ds.vocab, bs=64)

This does all the necessary preprocessing behing the scene. For the classifier, we also pass the vocabulary (mapping from ids to words) that we want to use: this is to ensure that data_clas will use the same dictionary as data_lm.



In [0]:
#Fine-tuning a language model
#We can use the data_lm object we created earlier to fine-tune a pretrained language model.
#fast.ai has an English model available that we can download. 
#We can create a learner object that will directly create a model, 
#download the pretrained weights and be ready for fine-tuning.
learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [0]:
# train the learner object
learn.fit_one_cycle(1, 1e-2) #29.99

In [0]:
learn.save_encoder('ft_enc_y1')

In [0]:
learn.lr_find()

In [0]:
learn.recorder.plot()

In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, 1e-2) # accuracy 36.48%

In [0]:
learn.save_encoder('ft_enc_y2')
#this times out, so fit one cycle 5+5 instead of 10 so we store encoder as well after 5 steps

In [0]:

learn.fit_one_cycle(2, 1e-2) 
learn.save_encoder('ft_enc_y3')# accuracy 37.12%
#Finally we save the encoder to be able to use it for classification in the next section.

In [0]:
learn.predict("This is a review about", n_words=15)

##Building a classifier
We now use the data_clas object we created earlier to build a classifier with our fine-tuned encoder. The learner object can be done in a single line.



In [0]:
learn.lr_find()
learn.recorder.plot()

In [0]:
#load the encoder models in models folder
#!mkdir models
#!mv ft_enc_y3.pth models/
#!mv tmp.pth models/

In [0]:
learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc_y3')

In [0]:
learn.fit_one_cycle(2, 1e-2)

In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(2, slice(5e-3/2., 5e-3))


In [0]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))


In [0]:
print("process completed")